## Obtain Credentials (not everytime)

Typically, it is necessary to periodically renew your credentials by calling the 'ee.Authenticate()' function. Upon calling this function, a web page will open where you can sign in with your Google account. This step allows Earth Engine to authenticate your identity and associate your API requests with your specific account. After the authentication process is successfully completed, a persistent token will be stored on your local machine. 
  
By calling the 'ee.Initialize()' function in either Step 1A or 1B, the saved credentials will be automatically retrieved, allowing you to make requests through the Earth Engine Python API. This grants you the necessary privileges to access various datasets and perform spatial analysis or processings.

In [4]:
import ee
ee.Authenticate()

Enter verification code: 4/1Adeu5BXoBgBE74q0-q1yHUfXI_fVfqLKK_6wXngjog3tdlFXvn3N13OzFm4

Successfully saved authorization token.


## Step 1A: Initialize with Regular Google Account

This is the way when you want to use Google Earth Engine with your regular google account, such as your gmail account. 

In [1]:
import ee
ee.Initialize()

import geemap
import Image
import LEAFNets as LFNs

## Step 2: Define Input Parameters

The execution of the LEAF production tool requires a number of parameters. To facilitate the handling and transfer of the parameters, a Python Dictionary is used as a container. Specifically, there are NINE key:value pairs in the dictionary: 
   
(1) 'sensor' : a string representing a satellite sensor and data unit (always surface reflectance). For exemple, 'S2_SR' and 'L8_SR' represent Sentinel-2 and Landsat-8 surface reflectance, respectively. 
   
(2) 'year' : a four-digit integer representing the year when the images were acquired, for instance, 2020.  
   
(3) 'months' : a list of month integers (e.g., [6,7,8]), enabling the generation of monthly biophysical products through a single execution of the LEAF production tool. each month integer in the list must fall within the valid range of 1 to 12. Otherwise, an invalid month integer in the list will cause a peak-season (June 15 to September 15) biophysical product is created.      
  
(4) 'tile_names' : a list of strings representing distinct tiles taht serve as the basic spatial unit within the LEAF production tool. Each tile encompasses an area of 900km x 900km as per the CCRS' tile griding system. By supplying a list of tile name strings, the tool can generate biophysical parameter products for multiple tiles in a single execution.  
     
(5) 'prod_names' : a list strings standing for different biophysical parameters. Currently, LEAF production tool can be used to generate a subset or a full set of FOUR biophysical parameters ['LAI', 'fCOVER', 'Albedo', 'fAPAR']. 
   
(6) 'resolution' : an integer defining the spatial resolution (in meter) of exported products.
   
(7) 'location' : a string that specifies the destination of exporting the products. This parameter accepts only two valid strings: 'drive' and 'storage', representing Google Drive and Google Cloud Storage, respectively. 
   
(8) 'bucket' : the name string of a bucket in Google Cloud Storage, which is required as a parameter only when the value for the 'location' key is set to 'storage'.
   
(9) 'folder' : the folder name string intended for exporting the products in either Google Drive or Google Cloud Storage. If this key is left empty, it indicates that a folder name will be generated automatically using the tile name and acquisition year.   
   
There are a total of nine parameters need to be defined. Out of these parameters, six of them (with 'sensor', 'year', 'resolution', 'location', 'bucket', and 'folder' as keys) require a single value, while the remaining parameters (with 'months', 'tile_names' and 'prod_names' as keys) need a list of values. By utilizing different combinations of these three lists, various production scenarios can be executed. For example, if the goal is to generate monthly parameter maps (e.g., for July and August) for multiple tiles (e.g., 'tile41', 'tile42', and 'tile43'), two lists need to be provided are [7,8] for 'months' key and ['tile41', 'tile42', 'tile43'] for 'tile_names' key.    

In [2]:
params = {
    'sensor': 'S2_SR',           # A string indicating sensor and data unit (e.g., 'S2_SR' or 'L8_SR')    
    'year': 2023,                # An integer representing image acquisition year
    'months': [7],               # A set of integers represening months (a negative value means whole summer)
    'tile_names': ['tile55'],    # A list of tile name strings as per the CCRS' tile griding system
    'prod_names': ['LAI','fAPAR','fCOVER','Albedo', 'QC', 'date'],   # ['LAI','fAPAR','fCOVER','Albedo', 'QC', 'date', 'partition']
    'resolution': 20,            # The spatial resultion (in meter) of the exported products
    'location': 'drive',         # The destination of exporting the products ('drive' or 'storage')     
    'bucket': 'S2_leaf_2023',    # An unique bucket name in GCS (must have been created beforehand)
    'folder': 'S2_leaf_2023_spec_score'     # the folder name intended for exporting products in either GD or GCS    
}


## Step 3: Generate and Export Biophysical Products

In [3]:
mosaic = LFNs.LEAF_production(params)


<LEAF_production> generate a global LC map.......
<Is_export_required>inquired str and prod_low: parti lai
<Is_export_required>inquired str and prod_low: parti fapar
<Is_export_required>inquired str and prod_low: parti fcover
<Is_export_required>inquired str and prod_low: parti albedo
<Is_export_required>inquired str and prod_low: parti qc
<Is_export_required>inquired str and prod_low: parti date

<LEAF_production> generate a mosaic for month 7
<getCollection> SsrData info: {'NAME': 'S2_SR', 'SSR_CODE': 21, 'DATA_UNIT': 2, 'GAIN': <ee.ee_number.Number object at 0x00000150C072A000>, 'OFFSET': <ee.ee_number.Number object at 0x00000150C072A030>, 'ALL_BANDS': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'], 'OUT_BANDS': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'], 'SIX_BANDS': ['B2', 'B3', 'B4', 'B8A', 'B11', 'B12'], 'NoA_BANDS': ['B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'], 'GEE_NAME': 'COPERNICUS/S2_SR_HARMONIZED', 'CLOUD': 'CLOUDY_

## Step 4: Check the Status of Exporting Tasks (optional)

Depending on whether you are using a regular Google account or a Google Service account to operate Google Earth Engine, there are two methods available to monitor the status of exporting tasks. If you're using a regular Google account, you can check the status by either accessing the "Tasks" tab located in the right panel of your Earth Engine Code Editor or executing the code provided in the subsequent cell. However, if you're using a Google Service account, the only way to monitor the status of exporting tasks is by running the code in the following cell.

The function "manage_tasks" requires two input parameters. The first parameter is a string and can take either 'status' or 'cancel' as its value. 'status' is used to check the status of exporting tasks, while 'cancel' is used to cancel all the exporting tasks. The second parameter is a string used to filter task names. For instance, providing '2019' will retrieve tasks with names that include '2019'. 

In [4]:
Image.manage_tasks('cancel', '2020')

tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m has been cancelled.
tile55_2020_Jul_fAPAR_200m has been cancelled.
tile55_2020_Jul_fCOVER_200m has been cancelled.
tile55_2020_Jul_LAI_200m has been cancelled.
tile55_2020_Jul_QC_200m has been cancelled.
tile55_2020_Jul_Albedo_200m 

## Step 1B: Initialize with Google Service Account

Initialze this way with your google service account.

In [2]:
import ee

from google.auth import compute_engine
credentials = compute_engine.Credentials(service_account_email = '751427818963-compute@developer.gserviceaccount.com', 
                                         scopes = ['https://www.googleapis.com/auth/earthengine'])

ee.Initialize(credentials, project='ccmeo-ag-000007')

import geemap
import Image
import LEAFNets as LFNs

ConnectionError: HTTPConnectionPool(host='metadata.google.internal', port=80): Max retries exceeded with url: /computeMetadata/v1/instance/service-accounts/751427818963-compute@developer.gserviceaccount.com/?recursive=true (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C887EED340>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))